In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import optimize
import crystal_mod as cry_mod 
import mode_analysis_code_original_fmm as ma_fmm

#create 3d array to hold data [ [ [x_p1_t0, x_p2_t0, x_p3_t0, ...],[y_p1_t0, y_p2_t0, ...], ... ],
#                              [ [x_p1_t1, x_p2_t1, x_p3_t1, ...],[y_p1_t1, y_p2_t1, ...], ... ],
#                                 . . .
#                              [ [x_p1_tn, x_p2_tn, x_p3_tn, ...],[y_p1_tn, y_p2_tn, ...], ... ] ]



#Open the binary file for reading
f = open("out_236i_2D_eq_evolve_writeeach_scratch.dat", "rb")

places=4
########## read number of particles (first number in data file) ##########
char_len = 1
char_empty = False
while not char_empty:
    f.seek(0)  #go to first character in file
    data_byte = f.read(char_len)
    num_particles = int(data_byte)
    char_len+=1
    next_char = f.read(1)
    if next_char ==  b" ":
        char_empty = True
        
print("Num particles: " + str(num_particles))

########## read number of recorded timesteps ##########
rec_steps = float(f.read(14))  #non-negative number in scientific notation has 12 characters
rec_steps = int(rec_steps)
print("Number of recorded steps: " + str(rec_steps))

f.read(1)  #read and ignore space

########## read number of total timesteps ##########
steps = float(f.read(12))  #non-negative number in scientific notation has 12 characters
steps = int(steps)
print("Number of steps: " + str(steps))

f.read(1)  #read and ignore space

########## read write rate ##########
write_per = float(f.read(12))  #non-negative number in scientific notation has 12 characters
write_per = int(write_per)
print("Write per: " + str(write_per))

f.read(1)  #read and ignore space

########## read duration ##########
duration = float(f.read(12))
print("Simulation duration: " + str(duration))

f.read(1)

########## read timestep ##########
dt = float(f.read(12))
print("Timestep: " + str(dt))

f.read(1)

########## read axial magnetic field ##########
data_byte = f.read(13+places) 
if float(data_byte) >= 0:  #if value is non-negative, only read 12 characters
    f.seek(-13-places,1)
    data_byte = f.read(12+places)
    Bz = float(data_byte)
else:  #if value is negative, read 13 characters
    Bz = float(data_byte)
f.read(1)

########## read k_z ##########
data_byte = f.read(13+places)
if float(data_byte) >= 0:
    f.seek(-13-places,1)
    data_byte = f.read(12+places)
    kz = float(data_byte)
else:
    kz = float(data_byte)
f.read(1)

########## read delta ##########
data_byte = f.read(13+places)
if float(data_byte) >= 0:
    f.seek(-13-places,1)
    data_byte = f.read(12+places)
    delta = float(data_byte)
else:
    delta = float(data_byte)
f.read(1)

########## read omega_R ##########
data_byte = f.read(13+places)
if float(data_byte) >= 0:
    f.seek(-13-places,1)
    data_byte = f.read(12+places)
    omegaR = float(data_byte)
else:
    omegaR = float(data_byte)
f.read(1)

########## read phi_0 ##########
data_byte = f.read(13+places)
if float(data_byte) >= 0:
    f.seek(-13-places,1)
    data_byte = f.read(12+places)
    phi0 = float(data_byte)
else:
    phi0 = float(data_byte)
f.read(1)

print("omegaR: " + str(omegaR))
print("Bz: " + str(Bz))
print("kz: " + str(kz))
print("delta: " + str(delta))
print("phi0: " + str(phi0))

########## read charges ##########
charges = np.array([])
for i in range(num_particles):
    data_byte = f.read(13+places)
    if float(data_byte) >= 0:
        f.seek(-13-places,1)
        data_byte = f.read(12+places)
        this_charge = float(data_byte)
        charges = np.append(charges, this_charge)
    else:
        this_charge = float(data_byte)
        charges = np.append(charges, this_charge)
    f.read(1)
    
########## read masses ##########
masses = np.array([])
for i in range(num_particles):
    data_byte = f.read(13+places)
    if float(data_byte) >= 0:
        f.seek(-13-places,1)
        data_byte = f.read(12+places)
        this_mass = float(data_byte)
        masses = np.append(masses, this_mass)
    else:
        this_mass = float(data_byte)
        masses = np.append(masses, this_mass)
        print("ERROR: NEGATIVE MASS")
    f.read(1)

print("Charges:")
print(charges)
print("Masses:")
print(masses)

####### beam parameters ############
char_len = 1
char_empty = False
num_beams_pos = f.tell()
while not char_empty:
    f.seek(num_beams_pos)  #go to first digit in num_beams
    data_byte = f.read(char_len)
    num_beams = int(data_byte)
    char_len+=1
    next_char = f.read(1)
    if next_char ==  b" ":
        char_empty = True
        
beam_S0 = np.array([]) #peak intensity
beam_khat = np.empty([3,num_beams]) #khat vectors of beams
beam_waist = np.array([]) #beam waists, -1 signifies uniform beam
beam_disp = np.empty([3, num_beams]) #beam displacement from origin
beam_det = np.array([]) #beam detuning from atomic transition

for i in range(num_beams):
    data_byte = f.read(13)
    if float(data_byte) >= 0:
        f.seek(-13,1)
        data_byte = f.read(12)
        this_S0 = float(data_byte)
        beam_S0 = np.append(beam_S0, this_S0)
    else:
        this_S0 = float(data_byte)
        beam_S0 = np.append(beam_S0, this_S0)
        print("ERROR: NEGATIVE INTENSITY")
    f.read(1)
    

for j in range(3):
    for i in range(num_beams):
        data_byte = f.read(13)
        if float(data_byte) >= 0:
            f.seek(-13,1)
            data_byte = f.read(12)
            this_k = float(data_byte)
            beam_khat[j][i] = this_k
        else:
            this_k = float(data_byte)
            beam_khat[j][i] = this_k
        f.read(1)
beam_khat = np.transpose(beam_khat)  #the ith row corresponds to the components of the khat vector for ith ion

for i in range(num_beams):
    data_byte = f.read(13)
    if float(data_byte) >= 0:
        f.seek(-13,1)
        data_byte = f.read(12)
        this_waist = float(data_byte)
        beam_waist = np.append(beam_waist, this_waist)
    else:
        this_waist = float(data_byte)
        beam_waist = np.append(beam_waist, this_waist)
    f.read(1)
'''
for i in range(num_beams):
    data_byte = f.read(13)
    if float(data_byte) >= 0:
        f.seek(-13,1)
        data_byte = f.read(12)
        this_disp = float(data_byte)
        beam_disp = np.append(beam_disp, this_disp)
    else:
        this_disp = float(data_byte)
        beam_disp = np.append(beam_disp, this_disp)
    f.read(1)'''
    
for j in range(3):
    for i in range(num_beams):
        data_byte = f.read(13)
        if float(data_byte) >= 0:
            f.seek(-13,1)
            data_byte = f.read(12)
            this_disp = float(data_byte)
            beam_disp[j][i] = this_disp
        else:
            this_disp = float(data_byte)
            beam_disp[j][i] = this_disp
        f.read(1)
beam_disp = np.transpose(beam_disp)  #the ith row corresponds to the components of the displacement vector for ith ion
    
for i in range(num_beams):
    data_byte = f.read(13)
    if float(data_byte) >= 0:
        f.seek(-13,1)
        data_byte = f.read(12)
        this_det = float(data_byte)
        beam_det = np.append(beam_det, this_det)
    else:
        this_det = float(data_byte)
        beam_det = np.append(beam_det, this_det)
    f.read(1)

print("Num beams: " + str(num_beams))
print("S0:")
print(beam_S0)
print("khat:")
print(beam_khat)
print("waist:")
print(beam_waist)
print("displacement:")
print(beam_disp)
#print(beam_disp[2][2])
print("detuning:")
print(beam_det)


############################## READ POS. & VEL. AT EACH TIMESTEP ##############################

time_counter = 0  #outermost index of data array, indexes timestep
type_counter = 0  #middle index which tells us to go to i.e. x positions array, or y velocities array, etc
val_counter = 0  #innermost index, indexes particle

times_rec = int((steps-1)/write_per)+2
times_rec = rec_steps
print("Times recorded: " + str(times_rec))
data = np.zeros((times_rec,6,num_particles))

while time_counter < times_rec:
    #read data for this timestep
    type_array = np.zeros((6, num_particles))  #has form [ [x_p1_t0, x_p2_t0, x_p3_t0, ...],[y_p1_t0, y_p2_t0, ...], ... ]
    while type_counter < 6:
        #for this data type, i.e. y-velocities, read data for each particle
        val_array = np.array([])  #has form [x_p1_t0, x_p2_t0, x_p3_t0, ...]
        while val_counter < num_particles:
            data_byte = f.read(13)
            #print(data_byte)
            if float(data_byte) >= 0 and str(float(data_byte))[0] != '-': #second coniditon is for -0.0e00
                #print(str(float(data_byte)))
                f.seek(-13,1)
                data_byte = f.read(12)
            
            this_val = float(data_byte)
            val_array = np.append(val_array, this_val)

            f.seek(1,1) #skip the space between values
            val_counter+=1
            
        type_array[type_counter] = val_array
        val_counter = 0
        type_counter+=1
    data[time_counter] = type_array
    type_counter=0
    time_counter+=1
    
print(data)
print(data.shape)
x_save = data[:,0,:]
y_save = data[:,1,:]
z_save = data[:,2,:]
vx_save = data[:,3,:]
vy_save = data[:,4,:]
vz_save = data[:,5,:]

#np.savetxt("x_test_100_dir2.csv", x_save, delimiter = ' ')
#np.savetxt("y_test_100_dir2.csv", y_save, delimiter = ' ')
#np.savetxt("z_test_100_dir2.csv", z_save, delimiter = ' ')
#np.savetxt("vx_test_100_dir2.csv", vx_save, delimiter = ' ')
#np.savetxt("vy_test_100_dir2.csv", vy_save, delimiter = ' ')
#np.savetxt("vz_test_100_dir2.csv", vz_save, delimiter = ' ')




















start_step = 0
end_step = times_rec
print(omegaR/(2*np.pi))
pos_rot = np.empty((end_step-start_step, 2, num_particles))
pos_rot3D = np.empty((end_step-start_step, 3, num_particles))
for i in range(start_step, end_step):
    this_x = np.array([])
    this_y = np.array([])
    this_z = np.array([])
    for j in range(num_particles):
        xx = np.cos(omegaR*dt*i*write_per)*data[i,0,j]+np.sin(omegaR*dt*i*write_per)*data[i,1,j] #rotation
        yy = -np.sin(omegaR*dt*i*write_per)*data[i,0,j]+np.cos(omegaR*dt*i*write_per)*data[i,1,j] #rotation
        this_x = np.append(this_x, xx)
        this_y = np.append(this_y, yy)
        this_z = np.append(this_z, data[i,2,j])
    pos_rot[i-start_step,0] =this_x
    pos_rot[i-start_step,1] =this_y
    pos_rot3D[i-start_step,0] =this_x
    pos_rot3D[i-start_step,1] =this_y
    pos_rot3D[i-start_step,2] =this_z
    
    
vel_rot3D = np.empty((end_step-start_step, 3, num_particles))
for i in range(start_step, end_step):
    this_vx = np.array([])
    this_vy = np.array([])
    this_vz = np.array([])
    for j in range(num_particles):
        velxx = -omegaR*np.sin(omegaR*dt*i*write_per)*data[i,0,j] + np.cos(omegaR*dt*i*write_per)*data[i,3,j] + omegaR*np.cos(omegaR*dt*i*write_per)*data[i,1,j] + np.sin(omegaR*dt*i*write_per)*data[i,4,j] #rotation
        velyy = -omegaR*np.cos(omegaR*dt*i*write_per)*data[i,0,j]-np.sin(omegaR*dt*i*write_per)*data[i,3,j] - omegaR*np.sin(omegaR*dt*i*write_per)*data[i,1,j]+np.cos(omegaR*dt*i*write_per)*data[i,4,j] #rotation
        this_vx = np.append(this_vx, velxx)
        this_vy = np.append(this_vy, velyy)
        this_vz = np.append(this_vz, data[i,5,j])
    vel_rot3D[i-start_step,0] =this_vx
    vel_rot3D[i-start_step,1] =this_vy
    vel_rot3D[i-start_step,2] =this_vz
    
    
    
    
    
    
    
    
    
write_per = 1
kb = 1.380649*10**-23
times_rec = 2000001
m = 9.012182*1.66057e-27

def Vel_arr(vels,num_particles,times_rec,write_per): # returns 2d array: index 1 is timestep and index 2 is mean particle KE
    ret_arr = np.zeros((times_rec, 2))
    for i in range(vels.shape[0]): #i indexes timestep
        this_vel = 0
        for j in range(num_particles):
            #this_energy += 0.5*masses[0]*(vel_rot3D[i,0,j]**2 +vel_rot3D[i,1,j]**2 +vel_rot3D[i,2,j]**2)
            this_vel+= np.sqrt(vels[i,0,j]**2+vels[i,1,j]**2+vels[i,2,j]**2)
            #this_vel= vel_rot3D[i,1,j]
        ret_arr[i, 1] = this_vel/num_particles
        ret_arr[i, 0] = i*write_per*dt
    return ret_arr

def temp_arr(vels,num_particles,times_rec,write_per): # returns 2d array: index 1 is timestep and index 2 is mean particle KE
    ret_arr = np.zeros((times_rec, 2))
    for i in range(vels.shape[0]): #i indexes timestep
        this_vel = 0
        for j in range(num_particles):
            #this_energy += 0.5*masses[0]*(vel_rot3D[i,0,j]**2 +vel_rot3D[i,1,j]**2 +vel_rot3D[i,2,j]**2)
            this_vel+= np.sqrt(vels[i,0,j]**2+vels[i,1,j]**2+vels[i,2,j]**2)
            #this_vel= vel_rot3D[i,1,j]
        ret_arr[i, 1] = (this_vel/num_particles)**2*masses[0]*np.pi/(8*kb)
        ret_arr[i, 0] = i*write_per*dt
    return ret_arr



Temp_arr = temp_arr(vel_rot3D, 236, 2000001,1)

Vel_arr1 = Vel_arr(vel_rot3D, 236, 2000001,1)

sfx = "236i"
np.savetxt("temp_" + sfx + ".csv", Temp_arr[:,1], delimiter = ' ')
np.savetxt("vel_" + sfx + ".csv", Vel_arr1[:,1], delimiter = ' ')

psx, freqs_normx = cry_mod.psd(pos_rot3D[:,0,:], 10**-9)
psy, freqs_normy = cry_mod.psd(pos_rot3D[:,1,:], 10**-9)
psz, freqs_normz = cry_mod.psd(pos_rot3D[:,2,:], 10**-9)
np.savetxt("psx_" + sfx + ".csv", psx, delimiter = ' ')
np.savetxt("freqs_normx" + sfx + ".csv", freqs_normx, delimiter = ' ')
np.savetxt("psy_" + sfx + ".csv", psy, delimiter = ' ')
np.savetxt("freqs_normy" + sfx + ".csv", freqs_normy, delimiter = ' ')
np.savetxt("psz_" + sfx + ".csv", psz, delimiter = ' ')
np.savetxt("freqs_normz" + sfx + ".csv", freqs_normz, delimiter = ' ')